### Write a program to convert a set of documents into tf-idf vectors

In [26]:
# Text collection: direction paths
import glob, re, os, nltk, math, string, collections, numpy
path = os.getcwd() #得到當前工作路徑 絕對
direc = glob.glob(path+"\IRTM\*.txt") #將該資料夾下所有檔案路徑回傳 #list

In [27]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

direc.sort(key=natural_keys)
#direc #這段到最後一天才發現有錯誤，文件順序跟最後寫的不同 #這一段參考資料來源: https://stackoverflow.com/questions/5967500/how-to-correctly-sort-a-string-with-a-number-inside

##### 1. Construct a dictionary based on the terms extracted from the given documents
* Record the document frequency of each term
* Save your dictionary as a txt file (dictionary.txt)

In [28]:
def tokenize(x): # x = file_path
    f = open(x, 'r')
    f = f.read()
    f = f.lower()
    f = f.replace('\n', '')
    f = f.replace('"', '')
    f = f.replace('\'', '')
    f = f.replace(',', '')
    f = f.replace('.', '')
    f = f.replace('?', '')
    f = f.replace('`', '')
    f = f.replace(';', '')
    f = f.replace(':', '')
    f = f.replace('(', '')
    f = f.replace(')', '')
    f = f.replace('_', '')
    f = f.replace('$', '')
    f = f.replace('-', '')
    f = f.replace('@', '')
    f = f.replace('%', '')
    f = f.replace('&amp', '')
    f = re.sub('\d', '', f)
    f = re.sub(r'\b\w\b', ' ', f) # 剔除一個字的
    f = re.sub(r'\b\w{2}\b', ' ', f) # 剔除兩個字的 i'm --> im --> x
    #f = re.sub(r'\b(http)(\\)(\w)\b', ' ', f) #把 url 刪掉, 後來覺得網址應該也是重要的，不用刪
    
    f = f.split() #以空白鍵分隔英文字
    
    #stemming
    from nltk.stem.porter import PorterStemmer
    pt = PorterStemmer()
    f1 = []
    for a in range(len(f)):
        f1.append(pt.stem(f[a]))
    
    #製作stopwordlist
    from nltk.corpus import stopwords
    stopwords_list = stopwords.words('english')
    #remove stopwords
    result = []
    for n in range(len(f1)):
        if f1[n] not in stopwords_list:
            result.append(f1[n])
    cnt = collections.Counter()
    for word in result:
        cnt[word] += 1
    return cnt

In [29]:
df = []
for a in range(len(direc)):
    df = df+list(tokenize(direc[a])) # count document term in collecitions
    cnt = collections.Counter()
    for word in df:
        cnt[word] += 1

In [30]:
dictionary = cnt.most_common()
len(dictionary)

13424

In [31]:
d1 = dict()
for a in range(len(dictionary)):
    d1.update({a+1: list(dictionary[a])})
import pandas as pd
import numpy as np
df = pd.DataFrame.from_dict(d1, orient='index')
df = df.rename(index=str, columns={0: 'term', 1:'df'})
df.index = np.arange(1, len(df) + 1)
df['t_index'] = df.index
df = df[['t_index', 'term', 'df']]
df

,t_index,term,df
1,1,wa,892
2,2,ha,753
3,3,said,729
4,4,hi,663
5,5,thi,626
6,6,say,593
7,7,one,590
8,8,state,570
9,9,two,569
10,10,presid,547


In [32]:
df.to_csv(path+"\dictionary.txt", header=None, index=None, sep='\t', mode='w')

##### 2. Transfer each document into a tf-idf unit vector
* Save it as a txt file (DocID.txt)

In [33]:
def transfer_doc_to_tfidf(file_path):
    # 把字典dictionary 讀進來，只需要1,2欄
    dic = pd.read_csv('dictionary.txt', sep = '\t', header = None, usecols = [1,2])
    dic = dic.rename(index=str, columns={1: 'term', 2:'df'}) #命名
    
    # 用字典來比對，字典包含文件x，則取出文件x的df
    doc_df = dic.loc[dic['term'].isin(list(tokenize(file_path)))] 
    doc_df = doc_df.reset_index()
    doc_df = doc_df.rename(index=str, columns={'index': 't_index'}) #命名
    
    # 計算idf, 輸入文件總數, df
    def idf_calculator(len_direc, doc_freq):
        idf = math.log10(len_direc/doc_freq)
        return idf
    idf = []
    for a in range(len(doc_df)):
            idf.append(idf_calculator(len(direc), doc_df.iloc[a]['df']))
    doc_df['idf'] = idf # 新增idf 這欄，以上面計算好的idf list 加上
    doc_df = doc_df.sort_values(by=['term'], ascending = True) # 以字順排序
    
    tf = tokenize(file_path).most_common()
    doc_tf = []
    for a in range(len(tf)):
        doc_tf.append(list(tf[a])) # 把tuple轉成 list
    doc_tf = pd.DataFrame(doc_tf) # 再轉成 dataframe
    doc_tf = doc_tf.rename(index=str, columns={0: 'term', 1:'tf'}) # 命名
    doc_tf = doc_tf.sort_values(by=['term'], ascending = True) # 以字順排序
    doc_df['tf'] = list(doc_tf['tf']) # 把 tf加上去 原本doc_df
    doc_df['tf_idf'] = doc_df.idf*doc_df.tf # 新增 tf_idf欄位，兩欄相乘
    
    #再補一個length normalize
    doc_df['Norm_tf_idf'] = doc_df.tf_idf/((sum(doc_df.tf_idf**2))**0.5)

    doc_df = doc_df.sort_values(by=['Norm_tf_idf'], ascending = False) # 以Norm_tf_idf大小排序，大到小
    
    return doc_df

In [34]:
# trytry看
transfer_doc_to_tfidf(direc[0])
#.to_csv(path+"\Doc001.txt", header=True, index=None, sep='\t', mode='w', columns=['t_index','Norm_tf_idf'])

,t_index,term,df,idf,tf,tf_idf,Norm_tf_idf
51,291,milosev,134,0.912309,6,5.473856,0.300925
66,461,belgrad,100,1.039414,5,5.197071,0.285709
75,555,strike,85,1.109995,4,4.439981,0.244088
121,2918,maceda,11,1.998021,2,3.996043,0.219683
40,176,opposit,186,0.769901,5,3.849506,0.211627
102,1173,tomorrow,41,1.426630,2,2.853261,0.156858
93,947,jim,51,1.331844,2,2.663688,0.146436
85,759,regim,64,1.233234,2,2.466468,0.135594
84,748,nbc,65,1.226501,2,2.453002,0.134854
129,4839,blackout,4,2.437354,1,2.437354,0.133994


In [35]:
for a in range(len(direc)):
    #補一個 文件 term 總數在開頭
    term_num = len(tokenize(direc[a]).most_common()) #This document has "term_num" terms.
    f = open(path+"\Doc"+str(a+1)+".txt", 'w')
    f.write('This document has ' + str(term_num) + ' terms. ' + '\n')

In [36]:
for a in range(len(direc)):
    transfer_doc_to_tfidf(direc[a]).to_csv(path+"\Doc"+str(a+1)+".txt", header=True, index=None, sep='\t', mode='a', columns=['t_index','term', 'Norm_tf_idf'])
    #mode = 'a' 往文件後面加上; sep = '\t' 以tab分隔

##### 3. Write a function cosine(Docx, Docy) which loads the tf-idf vectors of documents x and y and returns their cosine similarity

In [37]:
def cosin_sim(docx,docy): #輸入兩file_path
    # 首先把兩個檔讀進來 fx跟fy
    fa = pd.read_csv(docx, sep = '\t', header = 'infer', skiprows=1, usecols=[0,1,2])
    fb = pd.read_csv(docy, sep = '\t', header = 'infer', skiprows=1, usecols=[0,1,2])
    # 找到兩個都存在的term，另存成fm, fn
    fm = fb[fb['t_index'].isin(fa['t_index'])]
    fn = fa[fa['t_index'].isin(fb['t_index'])]
    # 依照t_index排序
    fm = fm.sort_values(by=['t_index'], ascending = True)
    fn = fn.sort_values(by=['t_index'], ascending = True)
    # 把t_index設成index, 如此才不會無法兩相對應
    fm = fm.set_index(['t_index'])
    fn = fn.set_index(['t_index'])
    # 新增一欄x, 把fm, fn兩個tf_idf相乘
    fm['x'] = fm.Norm_tf_idf*fn.Norm_tf_idf
    # 把上述欄相加，就是內積的結果
    cosin = sum(fm.x) 
    return cosin

In [38]:
cosin_sim('Doc1.txt', 'Doc2.txt')

0.2005967118514036

#### Please zip and submit
    1. your dictionary
    2. the vector file of document1
    3. source code
    4. a report to TA